# Python-1, Лекция 10

Лектор: Хайбулин Даниэль

Подготовил материал: Лущ Иван

Итак, сегодня мы поговорим про модули `collections`, `heapq` и `itertools`

### Collections

`collections` — это стандартный модуль Python, предоставляющий дополнительные структуры данных и функции, такие как `namedtuple`, `deque`, `Counter`, `OrderedDict`, `defaultdict` и другие. Эти структуры расширяют базовые возможности встроенных типов данных и позволяют более эффективно и удобно решать широкий спектр программных задач, связанных с обработкой и хранением данных.

На практике, как правило, наиболее часто используются такие структуры из модуля `collections`, как `defaultdict` и `namedtuple` (особенно актуален `namedtuple` при необходимости поддержки кода на `Python 2` или поддержки совместимости с `Python 2`).

#### defaultdict

`defaultdict` — это специализированный класс, который позволяет автоматически создавать и присваивать значения для отсутствующих ключей на основе заданной функции (или объекта, у которого определен *dunder-метод* `__call__`) по умолчанию (`default_factory`). При обращении к несуществующему ключу в таком словаре новый элемент автоматически добавляется и инициализируется значением, возвращаемым функцией по умолчанию, что избавляет от необходимости явно проверять наличие ключа или использовать метод `get`. Во всех остальных отношениях `defaultdict` полностью совместим с обычным словарём.

In [ ]:
from collections import defaultdict


dct: defaultdict[int, float] = defaultdict(float)

dct[2]

In [ ]:
from collections import defaultdict


dct: defaultdict[int, float] = defaultdict(list)

dct[2]

Если не передавать функцию по умолчанию (или передать `None`), `defaultdict` по сути ведёт себя как обычный `dict`: не создаёт новых элементов автоматически и выбрасывает `KeyError` при обращении к отсутствующему ключу.


Использование defaultdict без указания функции по умолчанию (default_factory=None) не рекомендуется, поскольку такой подход может ввести в заблуждение разработчиков, сопровождающих код.

In [ ]:
from collections import defaultdict


dct: dict[int, int] = defaultdict()

dct[2]

In [ ]:
from collections import defaultdict


dct: dict[int, int] = defaultdict(None)

dct[2]

##### Сравнение использования dict и defaultdict

In [ ]:
counter: dict[str, int] = dict()

elements: list[str] = ["apple", "banana", "apple", "orange", "banana", "apple"]

for fruit in elements:
    if fruit not in counter:
        counter[fruit] = 0
    counter[fruit] += 1

print(counter)

с использованием `get`:

In [ ]:
counter: dict[str, int] = dict()

elements: list[str] = ["apple", "banana", "apple", "orange", "banana", "apple"]

for fruit in elements:
    counter[fruit] = counter.get(fruit, 0) + 1

print(counter)

In [ ]:
from collections import defaultdict


# Создаем defaultdict с функцией (int) по умолчанию, которая возвращает 0
counter: defaultdict[str, int] = defaultdict(int)
# Можно еще написать так (это больше для понимая того, что во внутрь передается объект, который мы можем вызвать через круглые скобки), но такой код писать не стоит
# counter = defaultdict(lambda : 0)

elements: list[str] = ["apple", "banana", "apple", "orange", "banana", "apple"]

for fruit in elements:
    counter[fruit] += 1

print(counter)

С помощью `defaultdict` не требуется явно проверять наличие ключа или использовать метод `get()` для установки значения по умолчанию. Это уменьшает количество шаблонного кода и делает логику работы с данными более понятной.

##### Антипатерны

Не рекомендуется выполнять проверки наличия ключа в defaultdict посредством обращения к элементу по индексу. Такой подход может привести к неявному созданию новых элементов для отсутствующих ключей, что, в свою очередь, увеличит размер структуры данных и может повлиять на потребление памяти и производительность.

In [ ]:
# неправильно

dct: defaultdict[int, float] = defaultdict(float)

for i in range(5):
    if dct[i] == 0:
        print("Found")

dct

In [ ]:
# правильно

dct: defaultdict[int, float] = defaultdict(float)

for i in range(5):
    if i in dct:
        print("Found")

dct

Что не так с кодом? Какой будет вывод?

In [ ]:
# неправильно

lst = []
dct: defaultdict[int, list[int]] = defaultdict(lambda: lst)

for i in range(5):
    dct[i].append(1)

dct

In [ ]:
# правильно

dct: defaultdict[int, list[int]] = defaultdict(list)

for i in range(5):
    dct[i].append(1)

dct

#### namedtuple

`namedtuple` — это **функция** из модуля `collections`, позволяющая создавать неизменяемые (**immutable**) объекты-кортежи с именованными полями. Такая структура обеспечивает удобный доступ к полям как по имени, так и по индексу, сохраняя компактность и неизменяемость стандартных кортежей.

In [ ]:
from collections import namedtuple

Point = namedtuple("Point", ["x", "y"])
p1 = Point(2, 3)

print(p1.x)  # 2

# Как правило, при использовании namedtuple обращения к элементам по индексам не применяются,
# поскольку такой подход снижает читаемость и ясность кода.
# Вместо этого рекомендуется использовать доступ к значениям по именованным полям,
# что делает структуру данных более наглядной и облегчает сопровождение программы.
print(p1[1])  # 3

print(p1)  # Point(x=2, y=3)

# Нечитаемый код
result = p1[0] + p1[1]  # Не понятно, что тут x и y
print(result)

Еще раз хочется повториться, что `namedtuple` создает именно неизменяемые объекты

In [ ]:
Point = namedtuple("Point", ["x", "y"])
p = Point(1, 2)
p.x = 5

Начиная с `Python 3.7`, был добавлен модуль `dataclasses` для удобного создания классов данных. Классы, оформленные с помощью `@dataclass` могут быть как изменяемыми, так и неизменяемыми (при использовании параметра `frozen=True`). Чуть подробнее познакомимся с этим декоратором в дальнейших лекциях. На данном этапе не обязательно подробно разбираться в понятиях "декоратор" и "класс". Это темы, которые будут рассмотрены отдельными блоками позже, и их понимание не критично для начального освоения работы с `dataclass`.

In [ ]:
from dataclasses import dataclass


@dataclass(frozen=True)
class Point:
    x: int
    y: int


p2 = Point(2, 3)
print(p2.x)  # 2
print(p2)  # Point(x=2, y=3)


Из-за `frozen=True` не можем менять значения полей:

In [ ]:
p2.x = 3

В современных версиях Python (**начиная с 3.7**) рекомендуется отдавать предпочтение `dataclass` вместо `namedtuple` для создания структур данных. Это связано с тем, что `dataclass` обеспечивает большую гибкость, расширяемость и выразительность.

#### Counter

`Counter` — это специализированный класс, предназначенный для подсчёта количества вхождений элементов в итерируемых объектах (строках, списках и т.д.). По сути, это словарь, где ключами являются элементы, а значениями — количество их появлений.

In [ ]:
from collections import Counter


lst: list[str] = ["apple", "banana", "apple", "orange", "banana", "apple"]
c: Counter[str] = Counter(lst)
print(c)

подсчёт символов в строке:

In [ ]:
from collections import Counter


text = "abracadabra"
c: Counter[str] = Counter(text)
print(c)

Использование метода `most_common()`:

In [ ]:
from collections import Counter


words: list[str] = ["cat", "dog", "cat", "mouse", "dog", "cat"]
c: Counter[str] = Counter(words)
print(c.most_common(2))

Получение элементов с повторениями:

In [ ]:
from collections import Counter


c: Counter[str] = Counter(a=2, b=3, c=1)
print(list(c.elements()))

Так как `Counter` является специализированным подклассом стандартного словаря (`dict`), с объектами этого типа можно выполнять все основные операции, доступные для словарей. Это включает в себя обращение по ключу, изменение значений, итерирование по элементам, а также использование таких методов, как `items()`, `keys()`, `values()`, и других стандартных методов словаря. Таким образом, `Counter` полностью совместим с протоколом обычного словаря, что позволяет использовать его во всех контекстах, где ожидается стандартный словарь `Python`.

#### Deque

`Deque` — двусторонняя очередь. Является специализированной структурой данных, обеспечивающей эффективное добавление и удаление элементов с обоих концов очереди за константное время (`O(1)`). Это делает её предпочтительной для задач, связанных с реализацией стеков, очередей, а также сценариев с частым доступом к начальному и конечному элементам последовательности. В остальном `deque` поддерживает основные операции стандартных последовательностей Python.

* `Stack` - LIFO (last in first out) - гора тарелок

* `Queue` - FIFO (first in first out) - очередь в банке

* `Deque` - Двусторонняя очередь (можно добавлять и забирать как с конца, так и с начала)

С помощью deque можно создать двустороннюю очередь из любой последовательности, например из списка строк. После создания можем вывести все элементы очереди.

In [ ]:
from collections import deque


s: list[str] = ["ab", "ab", "bc", "cd", "ab"]
d: deque[str] = deque(s)
print(d)

print("-" * 30)

for elem in d:
    print(elem)

print("-" * 30)

Методы `append()` и `appendleft()` позволяют добавлять элементы в конец и начало очереди соответственно. С помощью .extend() и .extendleft() можно добавить несколько элементов сразу.

In [ ]:
d.append("aa")  # Добавление в конец
d.appendleft("dd")  # Добавление в начало
d.extend(["hh", "pp"])  # Добавление нескольких элементов в конец
d.extendleft(["hh", "pp"])  # Добавление нескольких элементов в начало

print(d)

Методы `pop()` и `popleft()` удаляют и возвращают элементы соответственно с правого и левого конца очереди.

In [ ]:
print(d.pop())  # Удаляет последний элемент
print(d.popleft())  # Удаляет первый элемент
print(d)

Можно обращаться к элементам по индексу, а также использовать методы `count()` для подсчета вхождений и `index()` для поиска индекса первого вхождения.

In [ ]:
print(d[0])  # Первый элемент
print(d[-2])  # Второй элемент с конца
print(d.count("hh"))  # Число вхождений 'hh'

Метод `rotate(n)` сдвигает элементы очереди на `n` позиций вправо (если `n` положительно) или влево (если `n` отрицательно). Метод `reverse()` разворачивает очередь.

In [ ]:
d.rotate(1)  # Сдвиг на 1 вправо
print(d)
d.rotate(-2)  # Сдвиг на 2 влево
print(d)
d.reverse()  # Разворот очереди
print(d)

Можно вставлять элементы по заданному индексу с помощью `insert()`, находить их позицию методом `index()`, а также удалять первое вхождение определенного значения с помощью `remove()`.

In [ ]:
d.insert(1, "x")  # Вставка элемента на позицию 1
print(d)
print(d.index("x"))  # Индекс элемента 'x'
d.remove("x")  # Удаление первого 'x'
print(d)

#### OrderedDict

`OrderedDict` — это подкласс стандартного словаря (`dict`) из модуля collections, который сохраняет порядок добавления элементов. То есть, при итерировании по `OrderedDict` элементы возвращаются именно в том порядке, в каком были добавлены. Кроме этого, `OrderedDict` предоставляет дополнительные методы, такие как `move_to_end() `и `popitem(last=True/False)`, которые позволяют манипулировать порядком элементов внутри структуры.

In [ ]:
from collections import OrderedDict


d: OrderedDict[str, int] = OrderedDict()
d["one"] = 1
d["two"] = 2
d["three"] = 3

for key, value in d.items():
    print(key, value)

Использование методов `move_to_end` и `popitem`:

In [ ]:
d.move_to_end("one")  # Перемещает 'one' в конец
print(list(d.keys()))  # ['two', 'three', 'one']

d.popitem(last=False)  # Убирает первый элемент ('two')
print(list(d.keys()))  # ['three', 'one']

d.popitem()  # Убирает последний элемент ('one'), по умолчанию last=True
print(list(d.keys()))  # ['three']

Начиная с версии `Python 3.7` (официально задокументировано в `Python 3.7`), стандартный `dict` сохраняет порядок добавления элементов точно так же, как это делал `OrderedDict`. Таким образом, во многих ситуациях надобность в использовании `OrderedDict` исчезла, и для большинства задач достаточно обычного словаря.

Когда все ещё полезен `OrderedDict`?

- Если требуются специальные методы (`move_to_end`, `popitem` с параметром). Например, при реализации логики `lru_cache`, очень удобно использовать именно `OrderedDict`. 
- Если важен явный контракт на поддержку порядка в старых версиях `Python (< 3.7)`.


#### heapq

`heapq` — это стандартный модуль в Python, предоставляющий набор функций для работы с кучей (heap), реализованной на базе обычного списка (`list`). Куча всегда хранит на первом месте (`heap[0]`) наименьший элемент. Основным назначением кучи являются эффективные операции получения и удаления минимальных (или максимальных, с некоторыми модификациями) элементов.


- `heapq.heapify(x)` — преобразует список x в кучу за время `O(n)`.
- `heapq.heappush(heap, item)` — добавляет элемент `item` в кучу.
- `heapq.heappop(heap)` — удаляет и возвращает наименьший элемент из кучи.
- `heapq.heappushpop(heap, item)` — вставляет элемент и возвращает наименьший за одну операцию.
- `heapq.heapreplace(heap, item)` — удаляет и возвращает наименьший, одновременно добавляя новый элемент.
- `heapq.nlargest(n, iterable)` — возвращает `n` наибольших элементов из последовательности.
- `heapq.nsmallest(n, iterable)` — возвращает `n` наименьших элементов.


In [ ]:
import heapq

# 1. Преобразование списка в кучу
lst: list[int] = [9, 3, 7, 1, 4]
heapq.heapify(lst)
print(lst)  # [1, 3, 7, 9, 4]

# 2. Добавление элемента
heapq.heappush(lst, 2)
print(lst)  # [1, 2, 7, 9, 4, 3]

# 3. Извлечение минимального значения
min_elem = heapq.heappop(lst)
print(min_elem)  # 1
print(lst)  # [2, 3, 7, 9, 4]

# 4. Одновременное добавление нового элемента и извлечение минимального
print(heapq.heappushpop(lst, 0))  # 0 (т.к. '0' меньше текущего минимума)
print(lst)  # [2, 3, 7, 9, 4]

# 5. Получение наибольших и наименьших элементов
nums = [2, 8, 5, 1, 6, 7]
print(heapq.nlargest(3, nums))  # [8, 7, 6]
print(heapq.nsmallest(2, nums))  # [1, 2]

### Itertools

`itertools` — это стандартный модуль **Python**, предоставляющий набор быстрых, эффективных и памяти экономных (ленивых) функций для создания и обработки итераторов.


В данном модуле представлен обширный набор функциональных возможностей, полный разбор которых выходит за рамки текущего материала. По собственному опыту могу отметить, что в практике разработки наиболее часто востребованы функции `chain` и `product`.

Полный перечень функций модуля приведён в официальной документации: https://docs.python.org/3/library/itertools.html. Ниже будут рассмотрены наиболее значимые, на взгляд автора, функции; с оставшимися вы, возможно, познакомитесь в ходе дальнейших занятий или самостоятельного изучения.

#### chain

In [ ]:
from itertools import chain

`itertools.chain(*iterables)` — это функция, которая принимает несколько итерируемых объектов (например, списки, строки, кортежи и т.д.) и возвращает единый итератор, проходящий по всем элементам этих итерируемых объектов последовательно, как если бы они были объединены в одну длинную последовательность.

In [ ]:
for e in chain(range(3), [1, 2], "lol", [[i] for i in range(4)]):
    print(e, end=" ")

В отличие от простого сложения списков `(a + b + c)`, chain работает с любыми итерируемыми объектами и не создает дополнительных коллекций в памяти.

Есть также вариант `chain.from_iterable(iterable_of_iterables)`, когда все последовательности заранее собраны в одном итерируемом объекте (например, в списке).

In [ ]:
iterables = [[1, 2], ("a", "b"), [3, 4]]
for x in chain.from_iterable(iterables):
    print(x)

#### product

In [ ]:
from itertools import product

`itertools.product(*iterables, repeat=1)` — это функция, возвращающая декартово произведение входных итерируемых последовательностей.

In [ ]:
roles: tuple[str, str, str] = ("bs", "yabs", "bsrank")
modes: tuple[str, str] = ("meta", "stat")

for r in roles:
    for m in modes:
        print(f"{r=}, {m=}")

Использование функции `product` повышает читаемость и ясность кода, позволяя явно выразить перебор всех возможных комбинаций элементов из заданных последовательностей без необходимости использования вложенных циклов:

In [ ]:
for r, m in product(roles, modes):
    print(f"{r=}, {m=}")

In [ ]:
a = product("AB", "12")
print(list(a))

#### islice, count, cycle, repeat

In [ ]:
from itertools import count, cycle, repeat, islice

`itertools.islice(iterable, start, stop[, step])` — это функция, позволяющая брать срезы с итерируемых объектов (в том числе бесконечных), подобных тому, как это делается с обычными списками.

Однако, в отличие от срезов списков (`sequence[start:stop:step]`), `islice` работает с любыми итерируемыми объектами (в том числе генераторами и потоками данных) и не требует загрузки всей последовательности в память.

In [ ]:
sequence = range(10)

In [ ]:
list(islice(sequence, 2, 5))  # sequence[2:5]

In [ ]:
list(islice(sequence, 2, 8, 2))  # sequence[2:8:2]

`itertools.count(start=0, step=1)` — это бесконечный итератор, который генерирует арифметическую последовательность чисел, начиная с значения `start` (по умолчанию `0`) и увеличивая (или уменьшая, если `step` отрицателен) предыдущее значение на величину `step` (по умолчанию `1`).

In [ ]:
i: count = count(10, 2)
print(next(i))
print(next(i))
print(next(i))
print(next(i))

list(islice(count(start=0, step=3), 10))

В отличие от `range`, `itertools.count()`:
- Бесконечный итератор: последовательность не имеет фиксированного конца и может продолжаться бесконечно (пока не остановить явно).
- Принимает любые значения для `start` и `step`, в том числе числа с плавающей запятой (`float`), отрицательные значения, ноль.
- Возвращает объект-итератор.

In [ ]:
from itertools import count

# count — бесконечная последовательность, можно использовать с любыми числами
for x in count(0.5, 0.5):
    print(x)
    if x >= 2:
        break


# range — конечная последовательность целых чисел
for i in range(1, 5, 1):
    print(i)

`itertools.cycle(iterable)` — это функция, возвращающая итератор, который бесконечно повторяет элементы переданной ему последовательности. После того, как все элементы iterable будут возвращены, перебор начинается сначала, и так далее без остановки.

In [ ]:
i: cycle = cycle("ABC")
print(next(i))
print(next(i))
print(next(i))
print(next(i))
print(next(i))
print(next(i))
print(next(i))
print(next(i))

In [ ]:
colors: list[str] = ["red", "green", "blue"]
cyclic_iter: cycle = cycle(colors)

# Получим первые 7 элементов циклического итератора
print(list(islice(cyclic_iter, 7)))

`itertools.repeat(object, times=None)` — это функция, возвращающая итератор, который выдаёт указанный объект столько раз, сколько задано в параметре `times`. Если параметр `times` не указан, итератор будет бесконечно возвращать объект.

In [ ]:
i: repeat = repeat("ABC", 5)
print(next(i))
print(next(i))
print(next(i))
print(next(i))
print(next(i))

In [ ]:
list(islice(repeat(42), 4))

#### permutations, combinations, combinations_with_replacement

In [ ]:
from itertools import permutations, combinations, combinations_with_replacement

`itertools.permutations(iterable, r)` возвращает все возможные перестановки длины `r` из элементов `iterable`. Порядок элементов важен, повторения не допускаются. Если `r` не указано, по умолчанию равно длине последовательности.

In [ ]:
a: permutations = permutations([1, 2, 3], r=3)
print(next(a))
print(next(a))
print(next(a))
print(next(a))
print(next(a))
print(next(a))

`itertools.combinations(iterable, r)` создаёт все уникальные комбинации длины `r` из элементов последовательности `iterable`. Порядок элементов в комбинации неважен, повторения не допускаются.

In [ ]:
a: combinations = combinations("ABACD", r=3)
print(next(a))
print(next(a))
print(next(a))
print(next(a))

`itertools.combinations_with_replacement(iterable, r)` возвращает все возможные комбинации длины `r` с повторениями из элементов `iterable`. Каждый элемент может входить в комбинацию несколько раз.

In [ ]:
a: combinations_with_replacement = combinations_with_replacement("ABCD", r=3)
print(next(a))
print(next(a))
print(next(a))
print(next(a))